## IMPORTING LIBRARIES AND DATA

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder  #for label encoding
import numpy as np
from numpy.linalg import norm
import pandas as pd
from sklearn.metrics.cluster import homogeneity_score
import sys
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#data = pd.read_csv("drive/MyDrive/SMAI_PROJECT_CLICKBAIT/train.csv", na_values='?', header=None,) 
data = pd.read_csv("clickbait-news-detection/train.csv", na_values='?', header=None,) 
data.head()
print(data.shape)
#print(data[2].isna().sum().sum())
#
df=data.dropna(axis = 0, how ='any')
print(df.shape)

(24872, 4)
(19878, 4)


In [4]:
data.describe


<bound method NDFrame.describe of            0                                                  1  \
0         id                                              title   
1          0  China and Economic Reform: Xi Jinping’s Track ...   
2          1  Trade to Be a Big Topic in Theresa May’s U.S. ...   
3          2  The Top Beaches In The World, According To Nat...   
4          3  Sheriff’s Report Provides New Details on Tamir...   
...      ...                                                ...   
24867  24866                                                NaN   
24868  24867  Unable to Enter U.S., and Still Stranded Abroa...   
24869  24868  Calais Migrant Camp Will Be Demolished Soon, F...   
24870  24869  Twitter’s NFL Deal No Cure for User and Advert...   
24871  24870  Five or Six Things I Didn’t Know About Brad Pi...   

                                                       2          3  
0                                                   text      label  
1      Economists gen

In [5]:
df=df.reset_index(drop=True)
df.describe

<bound method NDFrame.describe of            0                                                  1  \
0         id                                              title   
1          0  China and Economic Reform: Xi Jinping’s Track ...   
2          1  Trade to Be a Big Topic in Theresa May’s U.S. ...   
3          2  The Top Beaches In The World, According To Nat...   
4          3  Sheriff’s Report Provides New Details on Tamir...   
...      ...                                                ...   
19873  24864  A very interesting user generated content website   
19874  24867  Unable to Enter U.S., and Still Stranded Abroa...   
19875  24868  Calais Migrant Camp Will Be Demolished Soon, F...   
19876  24869  Twitter’s NFL Deal No Cure for User and Advert...   
19877  24870  Five or Six Things I Didn’t Know About Brad Pi...   

                                                       2          3  
0                                                   text      label  
1      Economists gen

#PREPROCESSING OF DATA

In [6]:
a=df.iloc[1:,2]
b=df.iloc[1:,1]

In [7]:
df.iloc[:,3] = df.iloc[:,3].str.replace('news','1')
df.iloc[:,3] = df.iloc[:,3].str.replace('clickbait','0')

y_actual=df.iloc[1:,3]


In [8]:
y_actual.describe

<bound method NDFrame.describe of 1            1
2            1
3            0
4            0
5            1
         ...  
19873    other
19874        1
19875        1
19876        1
19877        1
Name: 3, Length: 19877, dtype: object>

1. Converting all sentences to lower case 
2. Writing Abbreviations in full form
3. Removing punctuations (Normalisation)
4. Removing Stop words (Normalisation)
5. Lemmatizing the data



In [9]:
contractions = {
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have",
"'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have",
"didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
"hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
"he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
"i'd": "I would", "i'd've": "I would have", "i'll": "I will", "i'll've": "I will have",
"i'm": "I am", "i've": "I have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is",
"let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
"mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock",
"oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
"shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not",
"shouldn't've": "should not have", "so've": "so have", "so's": "so is", "that'd": "that had",
"that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have",
"there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
"wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
"we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
"what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is",
"what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
"where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why has", "why've": "why have",
"will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
"wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
"y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
"you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"
}


def clean_text(text):
    text = BeautifulSoup(ihtml.unescape(text)).text
    text = re.sub(r"http[s]?://\S+", "", text)
    text = re.sub(r"\s+", " ", text)    
    return text

In [10]:
#removing stop words
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

/bin/bash: pip: command not found


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/somyalalwani9/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/somyalalwani9/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/somyalalwani9/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
print(a)
print(a[41])
print(b)
print(b[41])

1        Economists generally agree: China must overhau...
2        LONDON—British Prime Minister Theresa May said...
3        Beaches come in all sorts of shapes and sizes ...
4        A timeline of what happened after Tamir Rice, ...
5        An Italian neurosurgeon who has claimed for mo...
                               ...                        
19873                           let me know what you think
19874    AMSTERDAM — Pedram Paragomi, a Iranian medical...
19875    (AFP) — The total dismantling of the “Jungle” ...
19876    Gaining the worldwide rights to stream 10 NFL ...
19877    The worst kind, too. The kind who lets a plant...
Name: 2, Length: 19877, dtype: object
A video has emerged of an 83-year-old man dancing energetically to music at an electronic dance festival in the Netherlands. In the footage, captured by a bystander at Edit Festival in Haarlem, Johan de Vries can be seen dancing in front of crowds of young revellers. Known locally as the ‘Grandfather of House’, 

In [12]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

final_lines=[]
final_headings=[]
for line in range(1,int(0.7*len(a))):
  #print(a[line])
  #print(type(a[line]))
  #print("*********")
  a[line]=(a[line]).lower()     #Converting all sentences to lower case 
  for word in a[line].split():
    if word in contractions:
        a[line]=a[line].replace(word, contractions[word.lower()])  #Writing Abbreviations in full form
  tokens = word_tokenize(a[line].lower()) 
  words = [word for word in tokens if word.isalpha()]    #Removing punctuations
  final_word = [w for w in words if not w in stop_words]     #Removing Stop words 
  final_words = [lemmatizer.lemmatize(w) for w in final_word]     #Lemmatizing words
  ans=""
  for x in final_words:
    ans= ans+ " "+x
  final_lines.append(ans.lstrip())
  b[line]=(b[line]).lower()     #Converting all sentences to lower case 
  for word in b[line].split():
    if word in contractions:
        b[line]=b[line].replace(word, contractions[word.lower()])  #Writing Abbreviations in full form
  tokens = word_tokenize(b[line].lower()) 
  words = [word for word in tokens if word.isalpha()]    #Removing punctuations
  final_head = [w for w in words if not w in stop_words]     #Removing Stop words 
  final_heads = [lemmatizer.lemmatize(w) for w in final_head]     #Lemmatizing words
  ans=""
  for x in final_heads:
    ans= ans+ " "+x
  final_headings.append(ans.lstrip())
#print(final_lines[:10])

In [13]:
#print(final_lines[679])
print(final_lines[0])
print(final_lines[1])
print(final_lines[2])

print(final_headings[0])
print(final_headings[1])
print(final_headings[2])

economist generally agree china must overhaul huge wasteful economy want continue grow year come mean limiting political interference banking financial system making bloated industry sensitive market force lowering barrier foreign trade investment proponent took heart late president xi jinping took formal control chinese communist party pledge crack corruption cut red tape today xi nears end first term much optimism among economist faded china remains heavily dependent large investment maintained brisk slowing economic growth steep rise lending still administration made small change hint xi may focus economic overhaul start second term xi done far important done reform expert give credit china move currency beijing persuaded international monetary fund admit currency renminbi elite club currency central bank hold reserve win approval china made much easier move money country shifted daily pegging renminbi dollar slightly system lingering problem move doubt lately opening money flow led

In [14]:
del data
del df
del tokens
del words
del final_word
del final_words
del final_head
del final_heads

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(final_lines)
Y = vectorizer.fit_transform(final_headings)
print("done")

done


In [16]:
print(type(X))
print(X[0])

<class 'scipy.sparse.csr.csr_matrix'>
  (0, 24799)	2
  (0, 32101)	1
  (0, 1498)	1
  (0, 14666)	13
  (0, 55419)	1
  (0, 60136)	2
  (0, 38070)	3
  (0, 90187)	1
  (0, 24800)	3
  (0, 90018)	1
  (0, 17386)	6
  (0, 34312)	3
  (0, 92715)	3
  (0, 16402)	1
  (0, 51801)	1
  (0, 47779)	1
  (0, 63709)	1
  (0, 40554)	1
  (0, 6488)	1
  (0, 29165)	3
  (0, 81284)	3
  (0, 49821)	2
  (0, 9204)	2
  (0, 39801)	1
  (0, 74002)	1
  :	:
  (0, 71309)	2
  (0, 4190)	1
  (0, 58492)	1
  (0, 69231)	3
  (0, 77055)	1
  (0, 76600)	1
  (0, 64643)	1
  (0, 55217)	1
  (0, 11603)	1
  (0, 59909)	1
  (0, 80036)	1
  (0, 61150)	1
  (0, 27549)	1
  (0, 1635)	1
  (0, 82741)	1
  (0, 11419)	1
  (0, 72441)	1
  (0, 4407)	1
  (0, 52635)	1
  (0, 13672)	1
  (0, 31394)	1
  (0, 91624)	1
  (0, 51909)	1
  (0, 40432)	1
  (0, 24933)	1


In [17]:
import numpy as np
#print(vectorizer.get_feature_names())
#final_word_vector = X.toarray()
#final_heading_vector= Y.toarray()


In [18]:
final_word_vector=[]

i=0
while (i<X.shape[0] and i+10000<X.shape[0]):
    final_word_vector.extend(X[i:i+10000].toarray())
    i+=10000

final_word_vector.extend(X[i:].toarray())


final_heading_vector=[]

i=0
while (i<Y.shape[0] and i+10000<Y.shape[0]):
    final_heading_vector.extend(Y[i:i+10000].toarray())
    i+=10000

final_heading_vector.extend(Y[i:].toarray())
print("done")

done


In [19]:
print(len(final_lines[0]))
#print(final_lines[0])
print(np.count_nonzero(final_word_vector[0]))
print(len(final_word_vector[0]))

4457
374
94049


In [20]:
print(len(final_lines[0]))
#print(final_lines[0])
print(np.count_nonzero(final_word_vector[0]))
print(len(final_word_vector[0]))

4457
374
94049


In [21]:
print(len(final_headings[0]))
#print(final_lines[0])
print(np.count_nonzero(final_heading_vector[0]))
print(len(final_heading_vector[0]))

45
7
15672


In [22]:
print(final_heading_vector.shape)
final_heading_vector = final_heading_vector.reshape(1, final_heading_vector.shape[0], final_heading_vector.shape[1])
print(final_heading_vector.shape)



AttributeError: 'list' object has no attribute 'shape'

In [ ]:
print(final_word_vector.shape)
final_word_vector = final_word_vector.reshape(1, final_word_vector.shape[0], final_word_vector.shape[1])
print(final_word_vector.shape)


In [ ]:
print(final_heading_vector.shape)
print(final_word_vector.shape)

In [ ]:
!pip3 install keras
!pip3 install tenserflow
!pip3 install --upgrade tensorflow

In [23]:
from tensorflow.keras.layers import LSTM, MaxPool1D, Dropout, Dense, GlobalMaxPooling1D, Embedding, Activation
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional


In [24]:

#!pip3 uninstall protobuf
#!pip3 uninstall google
#!pip3 install google
#!pip3 install protobuf
#!pip3 install google-cloud

import tensorflow


In [29]:
model = Sequential()
#model.add(Bidirectional(LSTM(2, return_sequences=True), input_shape=(13912,15672)))
#model.add(TimeDistributed(Dense(2, activation='sigmoid')))
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
"""
# train LSTM
for epoch in range(1000):
	# generate new random sequence
	X,y = get_sequence(n_timesteps)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, batch_size=1, verbose=2)
# evaluate LSTM
X,y = get_sequence(n_timesteps)
yhat = model.predict_classes(X, verbose=0)
for i in range(n_timesteps):
	print('Expected:', y[0, i], 'Predicted', yhat[0, i])
"""